In [5]:
import numpy as np
from ase import Atoms
from ase.build import bulk, mx2
from ase.io import write
from ase.visualize import view

def create_complex_structure():
    print("1. Создаем NaCl...")
    # ВАЖНО: cubic=True создает прямоугольную ячейку, а не наклонную.
    # Это предотвращает ошибку с нулевым размером массива.
    nacl = bulk('NaCl', 'rocksalt', a=5.64, cubic=True)
    nacl = nacl.repeat((4, 4, 4)) # Размер ~22x22x22 Å
    
    # Сдвигаем, чтобы начинался с 0
    nacl.translate(-nacl.positions[0])
    
    # Получаем целевые размеры (Y и Z), под которые будем подгонять другие слои
    target_y = nacl.cell[1, 1]
    target_z = nacl.cell[2, 2]
    print(f"   Размеры NaCl: {nacl.cell.lengths()}")

    print("2. Создаем Al...")
    # Тоже cubic=True
    al = bulk('Al', 'fcc', a=4.05, cubic=True)
    
    # Считаем повторения. max(1, ...) защищает от 0.
    rep_y = max(1, int(round(target_y / 4.05)))
    rep_z = max(1, int(round(target_z / 4.05)))
    rep_x = 4 # Толщина слоя Al
    
    al = al.repeat((rep_x, rep_y, rep_z))
    
    # Стыковка Al справа от NaCl
    gap_nacl_al = 2.5
    x_shift_al = nacl.positions[:, 0].max() + gap_nacl_al
    
    # Центровка по Y/Z
    dy = (target_y - al.cell[1, 1]) / 2
    dz = (target_z - al.cell[2, 2]) / 2
    
    al.translate((x_shift_al, dy, dz))
    print(f"   Размеры Al: {al.cell.lengths()}")

    print("3. Создаем MoS2 (2 слоя)...")
    # MoS2 (изначально плоский в XY)
    mos2 = mx2(formula='MoS2', kind='2H', a=3.16, thickness=3.172)
    
    # Сначала размножаем в плоскости
    lx, ly, _ = mos2.cell.lengths()
    rep_m_y = max(1, int(round(target_y / lx)))     # станет высотой Y
    rep_m_z = max(1, int(round(target_z / ly)))     # станет высотой Z
    
    mos2 = mos2.repeat((rep_m_y, rep_m_z, 1))
    
    # ПОВОРОТ: Делаем из "пола" "стену".
    # rotate(90, 'y') поворачивает оси: X->Z, Z->-X. Плоскость XY становится YZ.
    mos2.rotate(90, 'y')
    
    # Создаем второй слой
    mos2_layer2 = mos2.copy()
    mos2_layer2.translate((6.0, 0, 0)) # Сдвиг 6Å по новой оси X (бывшая Z)
    
    mos2_bilayer = mos2 + mos2_layer2
    
    # Стыковка MoS2 справа от Al
    gap_al_mos2 = 3.0
    # Ищем крайний атом алюминия
    x_shift_mos2 = al.positions[:, 0].max() + gap_al_mos2
    
    # Центровка MoS2
    # Берем центр масс коробки NaCl (Y и Z) и подгоняем MoS2 туда
    cm_nacl = nacl.get_center_of_mass()
    cm_mos2 = mos2_bilayer.get_center_of_mass()
    
    # Сдвигаем: по X ставим в позицию стыковки, по Y/Z выравниваем центры
    shift_vector = [
        x_shift_mos2 - cm_mos2[0], # Сдвигаем 'центр' MoS2 в нужную точку X? Нет, лучше просто сдвинуть край.
        cm_nacl[1] - cm_mos2[1],
        cm_nacl[2] - cm_mos2[2]
    ]
    
    # Коррекция по X (так как центр масс может быть где угодно, а нам нужен край)
    # Сначала просто применим выравнивание по Y Z
    mos2_bilayer.translate([0, shift_vector[1], shift_vector[2]])
    
    # Теперь точно выставим X. Находим текущий минимум X у MoS2
    current_min_x = mos2_bilayer.positions[:, 0].min()
    diff_x = x_shift_mos2 - current_min_x
    mos2_bilayer.translate((diff_x, 0, 0))

    print("4. Сборка...")
    structure = nacl + al + mos2_bilayer
    
    # Установка коробки (Cell)
    max_x = structure.positions[:, 0].max() + 15.0 # Вакуум
    structure.set_cell([max_x, target_y, target_z])
    structure.pbc = [False, True, True] # Периодичность по Y, Z
    
    return structure

# --- ЗАПУСК ---
atoms = create_complex_structure()
print(f"Готово! Всего атомов: {len(atoms)}")

# Сохранение (чтобы наверняка осталось)
write('final_structure.data', atoms, format='lammps-data', atom_style='atomic')
print("Сохранено в файл final_structure.data")

# Визуализация
# Если вы работаете на удаленном сервере без экрана, эта команда может выдать ошибку.
# Если локально - откроется окно.
try:
    view(atoms)
except Exception as e:
    print("Не удалось открыть графическое окно (возможно, нет дисплея). Используйте сохраненный файл.")

1. Создаем NaCl...
   Размеры NaCl: [22.56 22.56 22.56]
2. Создаем Al...
   Размеры Al: [16.2 24.3 24.3]
3. Создаем MoS2 (2 слоя)...
4. Сборка...
Готово! Всего атомов: 1382
Сохранено в файл final_structure.data


usage: ase [-h] [--version] [-T]
           {help,info,test,gui,db,run,band-structure,build,dimensionality,eos,ulm,find,nebplot,convert,reciprocal,completion,diff,exec}
           ...
ase: error: TclError: no display name and no $DISPLAY environment variable
To get a full traceback, use: ase -T gui ...


In [6]:
from ase.io import write
write('heterostructure.data', atoms, format='lammps-data', atom_style='atomic')

In [7]:
view(atoms)

<Popen: returncode: None args: ['/home/ebychkov/miniforge3/envs/HPPL_env/bin...>

usage: ase [-h] [--version] [-T]
           {help,info,test,gui,db,run,band-structure,build,dimensionality,eos,ulm,find,nebplot,convert,reciprocal,completion,diff,exec}
           ...
ase: error: TclError: no display name and no $DISPLAY environment variable
To get a full traceback, use: ase -T gui ...
